In [3]:
import pandas as pd
import numpy as np

data = pd.read_csv('./data/xgolf_contents_final.csv')
data # 데이터 확인

,Unnamed: 0,golf_name,title,id,datetime,year,month,day,time,golfscore,caddiescore,coursescore,pricescore,facility,contents
0,1,금강,굿 금강,yun****,2022.07.23 11:13,2022,7,23,11:13,10.0,10,10,10,10,페어웨이 그린 베스트 명품구장
1,2,비에이비스타퍼블릭,"양프로,티칭프로테스트를 앞둔 예비프로와 함께",Wls******,2022.07.23 11:08,2022,7,23,11:08,10.0,10,10,10,10,라운딩 체험으로 운동했는데 날씨도 햇빛 없는 날과함께 즐거웠습니다\n모든 좋았으나한...
2,3,몽베르,경기북부의 몽베르,NV2*******,2022.07.23 10:04,2022,7,23,10:04,10.0,10,10,10,10,편안하고 관리가 잘된 골프코스에서 라운딩했네요
3,4,스카이밸리,재촉만 하지 않았으면 최고였을텐데...,bab***,2022.07.22 23:11,2022,7,22,23:11,10.0,10,10,10,10,아침부터 비가 온 것을 제외하고는 좋았습니다.\n잘 관리된 페어웨이와 그린 그리고 ...
4,5,몽베르,몽베르라운딩후기,ljk******,2022.07.22 11:25,2022,7,22,11:25,10.0,10,10,10,10,첫홀은 비가와서 징행할까 말까 했었는데\n그래도 멀리왔는데 치고가자 고 일행이 적극...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47859,47860,기흥,좋은골프장 좋은그린,NV5******,2018.01.01 18:46,2018,1,1,18:46,10.0,10,10,10,10,겨울에도 기흥은 그린이 좋아요.속도도 맘에듭니다
47860,47861,아일랜드,아일랜드CC,dda****,2018.01.01 14:58,2018,1,1,14:58,10.0,10,10,10,10,바람이 좀 춥긴했지만 관리도 잘되있고 좋았습니다
47861,47862,기흥,눈 속에서,l3l***,2018.01.01 12:13,2018,1,1,12:13,9.5,10,10,8,10,눈이 오는 바람에 정상적인 라운드를 할 수 없었지만 너무 좋은 멤버들과 캐디 덕분에...
47862,47863,88,2017년 마지막 라운딩,pol*******,2018.01.01 04:17,2018,1,1,04:17,9.0,8,8,10,10,다행히 날씨가 춥지 않아 즐거운 라운딩이었음.\n11:32 티업. 전반 지나고 그늘...


In [19]:
data.info() # 개괄 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47864 entries, 0 to 47863
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   47864 non-null  int64  
 1   golf_name    47864 non-null  object 
 2   title        47864 non-null  object 
 3   id           47864 non-null  object 
 4   datetime     47864 non-null  object 
 5   year         47864 non-null  int64  
 6   month        47864 non-null  int64  
 7   day          47864 non-null  int64  
 8   time         47864 non-null  object 
 9   golfscore    47864 non-null  float64
 10  caddiescore  47864 non-null  int64  
 11  coursescore  47864 non-null  int64  
 12  pricescore   47864 non-null  int64  
 13  facility     47864 non-null  int64  
 14  contents     47864 non-null  object 
dtypes: float64(1), int64(8), object(6)
memory usage: 5.5+ MB


In [20]:
data['id'].nunique() # unique id 개수 확인

6140

In [21]:
data['id'].value_counts() # id별 리뷰 개수 확인(소셜로그인 계정 - NV: Naver, KK: Kakao)

NV2*******      1159
NV3*******      1002
KK1*********     811
NV1*******       774
NV4*******       724
                ... 
dar*******         1
uam**              1
g08**              1
khv****            1
chz*****           1
Name: id, Length: 6140, dtype: int64

In [22]:
# 소셜로그인 계정 리뷰 삭제(과도한 중복 피하기)
data['id'][~((data['id'].str.startswith('NV')) | (data['id'].str.startswith('KK')))].nunique()

6117

In [23]:
# 소셜로그인 계정 제외한 id별 리뷰 개수 확인
data['id'][~((data['id'].str.startswith('NV')) | (data['id'].str.startswith('KK')))].value_counts()

act*****       609
shi*****       177
moo*****       157
lee****        136
hip******      130
              ... 
put***           1
lhj****          1
khv****          1
jja********      1
den****          1
Name: id, Length: 6117, dtype: int64

In [24]:
# 소셜로그인 계정 제외한 전체 리뷰 개수 확인
data['id'][~((data['id'].str.startswith('NV')) | (data['id'].str.startswith('KK')))].count()

41361

In [25]:
data_1 = data[~((data['id'].str.startswith('NV')) | (data['id'].str.startswith('KK')))]
data_1

,Unnamed: 0,golf_name,title,id,datetime,year,month,day,time,golfscore,caddiescore,coursescore,pricescore,facility,contents
0,1,금강,굿 금강,yun****,2022.07.23 11:13,2022,7,23,11:13,10.0,10,10,10,10,페어웨이 그린 베스트 명품구장
1,2,비에이비스타퍼블릭,"양프로,티칭프로테스트를 앞둔 예비프로와 함께",Wls******,2022.07.23 11:08,2022,7,23,11:08,10.0,10,10,10,10,라운딩 체험으로 운동했는데 날씨도 햇빛 없는 날과함께 즐거웠습니다\n모든 좋았으나한...
3,4,스카이밸리,재촉만 하지 않았으면 최고였을텐데...,bab***,2022.07.22 23:11,2022,7,22,23:11,10.0,10,10,10,10,아침부터 비가 온 것을 제외하고는 좋았습니다.\n잘 관리된 페어웨이와 그린 그리고 ...
4,5,몽베르,몽베르라운딩후기,ljk******,2022.07.22 11:25,2022,7,22,11:25,10.0,10,10,10,10,첫홀은 비가와서 징행할까 말까 했었는데\n그래도 멀리왔는데 치고가자 고 일행이 적극...
5,6,썬힐,정말 해도해도 너무하네요,cli*****,2022.07.22 09:30,2022,7,22,09:30,3.5,8,2,2,2,좋은 날 기분 더럽게 다녀왔네요.. 코스관리 개엉망이고 그린은 완전 걸레짝. 카운터...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47856,47857,스카이밸리,마운틴코스라기에. . .,ana*****,2018.01.01 21:50,2018,1,1,21:50,9.5,10,8,10,10,눈이라도 많을까봐 걱정했는데.\n플레이하기에 전혀 지장이 없는\n상태로 코스는 잘 ...
47860,47861,아일랜드,아일랜드CC,dda****,2018.01.01 14:58,2018,1,1,14:58,10.0,10,10,10,10,바람이 좀 춥긴했지만 관리도 잘되있고 좋았습니다
47861,47862,기흥,눈 속에서,l3l***,2018.01.01 12:13,2018,1,1,12:13,9.5,10,10,8,10,눈이 오는 바람에 정상적인 라운드를 할 수 없었지만 너무 좋은 멤버들과 캐디 덕분에...
47862,47863,88,2017년 마지막 라운딩,pol*******,2018.01.01 04:17,2018,1,1,04:17,9.0,8,8,10,10,다행히 날씨가 춥지 않아 즐거운 라운딩이었음.\n11:32 티업. 전반 지나고 그늘...


In [26]:
data_1 = data_1.drop(columns='Unnamed: 0').reset_index(drop=True)
data_1.rename(columns={'golf_name':'cc_name', 'golfscore':'cc_score', 'caddiescore':'caddie', 'coursescore':'course', 'pricescore': 'price'}, inplace=True)
data_1 # base df

,cc_name,title,id,datetime,year,month,day,time,cc_score,caddie,course,price,facility,contents
0,금강,굿 금강,yun****,2022.07.23 11:13,2022,7,23,11:13,10.0,10,10,10,10,페어웨이 그린 베스트 명품구장
1,비에이비스타퍼블릭,"양프로,티칭프로테스트를 앞둔 예비프로와 함께",Wls******,2022.07.23 11:08,2022,7,23,11:08,10.0,10,10,10,10,라운딩 체험으로 운동했는데 날씨도 햇빛 없는 날과함께 즐거웠습니다\n모든 좋았으나한...
2,스카이밸리,재촉만 하지 않았으면 최고였을텐데...,bab***,2022.07.22 23:11,2022,7,22,23:11,10.0,10,10,10,10,아침부터 비가 온 것을 제외하고는 좋았습니다.\n잘 관리된 페어웨이와 그린 그리고 ...
3,몽베르,몽베르라운딩후기,ljk******,2022.07.22 11:25,2022,7,22,11:25,10.0,10,10,10,10,첫홀은 비가와서 징행할까 말까 했었는데\n그래도 멀리왔는데 치고가자 고 일행이 적극...
4,썬힐,정말 해도해도 너무하네요,cli*****,2022.07.22 09:30,2022,7,22,09:30,3.5,8,2,2,2,좋은 날 기분 더럽게 다녀왔네요.. 코스관리 개엉망이고 그린은 완전 걸레짝. 카운터...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41356,스카이밸리,마운틴코스라기에. . .,ana*****,2018.01.01 21:50,2018,1,1,21:50,9.5,10,8,10,10,눈이라도 많을까봐 걱정했는데.\n플레이하기에 전혀 지장이 없는\n상태로 코스는 잘 ...
41357,아일랜드,아일랜드CC,dda****,2018.01.01 14:58,2018,1,1,14:58,10.0,10,10,10,10,바람이 좀 춥긴했지만 관리도 잘되있고 좋았습니다
41358,기흥,눈 속에서,l3l***,2018.01.01 12:13,2018,1,1,12:13,9.5,10,10,8,10,눈이 오는 바람에 정상적인 라운드를 할 수 없었지만 너무 좋은 멤버들과 캐디 덕분에...
41359,88,2017년 마지막 라운딩,pol*******,2018.01.01 04:17,2018,1,1,04:17,9.0,8,8,10,10,다행히 날씨가 춥지 않아 즐거운 라운딩이었음.\n11:32 티업. 전반 지나고 그늘...


In [27]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41361 entries, 0 to 41360
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   cc_name   41361 non-null  object 
 1   title     41361 non-null  object 
 2   id        41361 non-null  object 
 3   datetime  41361 non-null  object 
 4   year      41361 non-null  int64  
 5   month     41361 non-null  int64  
 6   day       41361 non-null  int64  
 7   time      41361 non-null  object 
 8   cc_score  41361 non-null  float64
 9   caddie    41361 non-null  int64  
 10  course    41361 non-null  int64  
 11  price     41361 non-null  int64  
 12  facility  41361 non-null  int64  
 13  contents  41361 non-null  object 
dtypes: float64(1), int64(7), object(6)
memory usage: 4.4+ MB


### 사용자 데이터프레임 생성

In [28]:
data_1['id'].nunique() # user count 확인

6117

In [29]:
data_id = data_1['id'].sort_values().drop_duplicates().reset_index()
data_id = data_id.rename(columns={'index':'id_num'})
data_id['id_num'] = range(1,len(data_id)+1)
data_id # user df

,id_num,id
0,1,002***
1,2,003****
2,3,007***
3,4,007*****
4,5,010*******
...,...,...
6112,6113,zzi****
6113,6114,zzi*********
6114,6115,zzu****
6115,6116,zzz*


### 골프장 데이터프레임 생성

In [30]:
data_cc = data_1['cc_name'].sort_values().drop_duplicates().reset_index()
data_cc

,index,cc_name
0,3847,360도
1,1847,88
2,3240,H1 CLUB(구 덕평)
3,31750,XGOLF
4,10553,XGOLF(테스트)
...,...,...
134,34854,한양파인(P9)
135,41191,한원
136,6717,해솔리아
137,32543,화성(P9)


In [31]:
# XGOLF 삭제
index_drop = data_1[data_1['cc_name'].str.contains('XGOLF')].index
data_1.drop(index_drop, inplace=True)

In [32]:
pd.set_option('display.max_rows', None) ## 모든 행 출력

data_cc = data_1['cc_name'].sort_values().drop_duplicates().reset_index()
data_cc = data_cc.rename(columns={'index':'cc_num'})
data_cc['cc_num'] = range(1,len(data_cc)+1)
data_cc

,cc_num,cc_name
0,1,360도
1,2,88
2,3,H1 CLUB(구 덕평)
3,4,가산노블리스
4,5,가평베네스트
5,6,강남300
6,7,고양컨트리클럽(P9)
7,8,곤지암
8,9,골드
9,10,골프존카운티안성H


In [33]:
pd.set_option('display.max_rows', 50) ## 원복
data_cc.shape

(137, 2)